In [ ]:

awk '{key=$1":"$4; a1[$2]=$5; a2[$2]=$6; pos[key]=$2} END {
     for (k in pos) print k,pos[k],a1[pos[k]],a2[pos[k]];
}' MBBRAINS_R123-HumanCoreExome-12v1_chrall_rsq9_hrc1.1.bim | head


# Set Up 
plink --bfile MBBRAINS_R123-HumanCoreExome-12v1_chrall_rsq9_hrc1.1 \
      --freq \
      --out cohort_freq

awk 'NR>1 {a1_frq[$2]=$3; a2_frq[$2]=$4; maf[$2]=$5; next}
     $5=="ADD" {
         snp=$2; chr=$1; pos=$3; a1=$4; beta=$7; stat=$9;
         if (stat == 0 || stat == "NA") {
             se="NA"; z="NA";
         } else {
             se=beta/stat; z=stat;
         }
         print snp,chr,pos,a1,a2_frq[snp],maf[snp],beta,se,z;
     }' cohort_freq.frq GWAS_rsq9_hrc1_alltraits.P1.assoc.linear > region_all.body

sed -i '1isnp chrom pos a1 a2 maf beta se z' region_all.body
mv region_all.body region_all.z

# Calculate SE and Z
awk 'NR==1 {print "snp chrom pos a1 a2 maf beta se z"; next}
     NR>1 {
         se = ($7=="NA" || $7==0 ? "NA" : $6/$7);
         print $1,$2,$3,$4,$5,$6,se,$7
     }' gwas_with_a2.tmp > region_all.z

In [ ]:
# P1_fixed.v

cat > create_zfile_fixed.sh << 'EOF'
#!/bin/bash
# create_zfile_fixed.sh <GWAS_FILE> <BIM_FILE> <FRQ_FILE> <OUTPUT_Z>

GWAS=$1
BIM=$2
FRQ=$3
OUTPUT=$4

# Pre-load all necessary data
echo "Loading data into memory..."

# Create mapping from chr:pos to rsID and alleles
awk '{print $1":"$4, $2, $5, $6}' "$BIM" | tr -d '\r' > full_bim_cache.txt

# Create FRQ cache
awk 'NR>1 {print $2, $5}' "$FRQ" | tr -d '\r' > frq_cache.txt

awk -v OFS='\t' '
BEGIN {
    print "snp", "chrom", "pos", "a1", "a2", "maf", "beta", "se", "z"

    # Load full BIM cache
    while ((getline line < "full_bim_cache.txt") > 0) {
        split(line, parts, " ")
        chrpos = parts[1]
        rsid = parts[2]
        a1_val = parts[3]
        a2_val = parts[4]
        
        # Store mappings
        chrpos_to_rsid[chrpos] = rsid
        chrpos_to_a1[chrpos] = a1_val
        chrpos_to_a2[chrpos] = a2_val
        rsid_to_a1[rsid] = a1_val
        rsid_to_a2[rsid] = a2_val
    }
    close("full_bim_cache.txt")

    # Load FRQ cache
    while ((getline line < "frq_cache.txt") > 0) {
        split(line, parts, " ")
        maf_by_rsid[parts[1]] = parts[2]
    }
    close("frq_cache.txt")
}

NR>1 && $5=="ADD" {
    original_snp_id = $2
    chrom = $1
    pos = $3
    beta = $7
    se = $8
    sub(/-.*/, "", se)  # Clean SE column
    z = $11

    # Initialize variables
    a1 = "NA"
    a2 = "NA" 
    maf = "NA"
    output_snp_id = original_snp_id

    # Determine SNP type and process accordingly
    if (original_snp_id ~ /^rs/) {
        # rsID format
        if (original_snp_id in rsid_to_a1) {
            a1 = rsid_to_a1[original_snp_id]
            a2 = rsid_to_a2[original_snp_id]
        }
        if (original_snp_id in maf_by_rsid) {
            maf = maf_by_rsid[original_snp_id]
        }
    } else {
        # chr:pos format - extract clean chr:pos
        clean_snp_id = original_snp_id
        gsub("chr", "", clean_snp_id)
        split(clean_snp_id, parts, /[:-]/)
        chr_pos = parts[1] ":" parts[2]
        
        if (chr_pos in chrpos_to_a1) {
            a1 = chrpos_to_a1[chr_pos]
            a2 = chrpos_to_a2[chr_pos]
        }
        
        # Get rsID for MAF lookup
        if (chr_pos in chrpos_to_rsid) {
            rsid = chrpos_to_rsid[chr_pos]
            if (rsid in maf_by_rsid) {
                maf = maf_by_rsid[rsid]
            }
        }
    }

    print original_snp_id, chrom, pos, a1, a2, maf, beta, se, z
}
' "$GWAS" > "$OUTPUT"

# Clean up
rm -f full_bim_cache.txt frq_cache.txt
EOF

chmod +x create_zfile_fixed.sh

# Run the fixed version
./create_zfile_fixed.sh \
  GWAS_P1_rsid.assoc.linear \
  MBBRAINS_R123-HumanCoreExome-12v1_chrall_rsq9_hrc1.1.bim \
  cohort_freq.frq \
  P1_fixed.z

# Check results
head -n 10 P1_fixed.z

In [ ]:
# Run_finemap_regions.sh
#!/bin/bash
# run_finemap_regions.sh <Zfile> <BEDfile> <BFILE> <OUTDIR>
# Example: bash run_finemap_regions.sh P1.z P1_regions_250kb.bed MBBRAINS_R123-HumanCoreExome-12v1_chrall_rsq9_hrc1.1 finemap_P1

ZFILE=$1
BED=$2
BFILE=$3
OUTDIR=$4

mkdir -p $OUTDIR

i=0
while read chr start end name; do
  i=$((i+1))
  prefix="${OUTDIR}/region_${i}_${name}"

  echo ">>> Processing $chr:$start-$end ($name)"

  # Extract SNPs from BIM inside region
  awk -v c=$chr -v s=$start -v e=$end 'BEGIN{OFS="\t"} {
      if("chr"$1==c && $4>=s && $4<=e) print $2
  }' ${BFILE}.bim > ${prefix}.snps

  # Subset Z
  awk 'NR==FNR{a[$1]=1; next} NR==1 || ($1 in a)' ${prefix}.snps $ZFILE > ${prefix}.z   

  # Compute LD matrix
  plink --bfile $BFILE \
        --r square gz \
        --extract ${prefix}.snps \
        --out ${prefix}.ld

  # Write master file

awk 'NR>1 {print $1}' ${prefix}.z > ${prefix}.snp

cat > "${TAG}.master" <<EOF
z;ld;snp;config;cred;log;n_samples
${TAG}.z;${TAG}.ld;${TAG}.snp;${TAG}.config;${TAG}.cred;${TAG}.log;${N}
EOF

  # Run FINEMAP
 ./finemap_v1.4_x86_64/finemap_v1.4_x86_64 --sss --in-files ${prefix}.master --n-causal-snps 3 --log

done < $BED

In [ ]:
# Run_finemap_regions.sh (V2)

#!/bin/bash
# Usage: bash run_finemap_regions.sh <Zfile> <BEDfile> <BFILE_PREFIX> <OUTDIR> [N_SAMPLES]
# Zfile must have header: snp chrom pos a1 a2 maf beta se z
# BEDfile must have 4 cols: chrom start end rsid  (chrom can be '2' or 'chr2')
set -euo pipefail

if [ $# -lt 4 ]; then
  echo "Usage: $0 ZFILE BEDFILE BFILE OUTDIR [N_SAMPLES]" >&2
  exit 1
fi

ZFILE=$1
BED=$2
BFILE=$3      # e.g. MBBRAINS_R123-..._chrall_rsq9_hrc1.1  (no extension)
OUTDIR=$4
N=${5:-256}   # default n_samples; override with 5th arg if you want

mkdir -p "$OUTDIR"

# Ensure PLINK is in PATH
if ! command -v plink >/dev/null 2>&1; then
  if command -v module >/dev/null 2>&1; then module load plink/1.90 || true; fi
fi
command -v plink >/dev/null 2>&1 || { echo "ERROR: plink not found in PATH."; exit 1; } 

i=0
while read -r chr start end name; do
  i=$((i+1))
  tag="${OUTDIR}/region_${i}_${name}"
  echo ">>> Processing ${chr}:${start}-${end} (${name})"

  # Normalize chromosome (accept '2' or 'chr2')
  chr_num="${chr#chr}"

  # 1) Slice Z by coordinates
  awk -F'\t' -v c="$chr_num" -v s="$start" -v e="$end" \
      'NR==1 || ($2==c && $3>=s && $3<=e)' "$ZFILE" > "${tag}.z"

  nvars=$(awk 'END{print NR-1}' "${tag}.z")
  if [ "$nvars" -le 0 ]; then
    echo "WARNING: no Z variants in ${chr}:${start}-${end}; skipping."
    rm -f "${tag}.z"
    continue
  fi

  # 2) LD matrix for the same region (plain .ld) + write PLINK snplist (order)
  plink --bfile "$BFILE" \
        --chr "$chr_num" --from-bp "$start" --to-bp "$end" \
        --r square --write-snplist \
        --out "${tag}"

  # If PLINK found zero variants in reference, skip
  if [ ! -s "${tag}.ld" ] || [ ! -s "${tag}.snplist" ]; then
    echo "WARNING: no reference variants for LD in ${chr}:${start}-${end}; skipping."   
    rm -f "${tag}.z" "${tag}.ld" "${tag}.snplist"
    continue
  fi

  # 3) Reorder the Z slice to PLINK's snplist order (and drop SNPs not in LD)
  awk 'NR==FNR {ord[++n]=$1; next}
       NR==1 {hdr=$0; next}
       {row[$1]=$0}
       END {
         print hdr;
         for(i=1;i<=n;i++) if (ord[i] in row) print row[ord[i]];
       }' "${tag}.snplist" "${tag}.z" > "${tag}.z.reordered"
  mv "${tag}.z.reordered" "${tag}.z"

  # 4) Master file (header + one dataset line, including n_samples)
  cat > "${tag}.master" <<EOF
z;ld;snp;config;cred;log;n_samples
${tag}.z;${tag}.ld;${tag}.snp;${tag}.config;${tag}.cred;${tag}.log;${N}
EOF

  # 5) Run FINEMAP
  ./finemap_v1.4_x86_64/finemap_v1.4_x86_64 --sss --in-files "${tag}.master" --n-causal-snps 3 --log

done < "$BED"


In [ ]:
# BACKUP the current per-region z's (in case you want to inspect)
mkdir -p finemap_P1/_backup_z && cp finemap_P1/region_*_*.z finemap_P1/_backup_z/ 2>/dev/null

# Rebuild each region z by coordinate filter + intersect with PLINK snplist + safe SE
i=0
while read chr start end name; do
  i=$((i+1))
  prefix="finemap_P1/region_${i}_${name}"

  # If you already created .snplist via PLINK, great; if not, this makes it:
  # plink --bfile MBBRAINS_R123-HumanCoreExome-12v1_chrall_rsq9_hrc1.1 \
  #       --chr $chr --from-bp $start --to-bp $end \
  #       --write-snplist --r square --out "$prefix"

  awk -v OFS="\t" -v C="$chr" -v S="$start" -v E="$end" '
    FNR==NR { keep[$1]=1; next }                                # snplist
    NR==1 { print "rsid","chromosome","position","allele1","allele2","maf","beta","se"; next }
    # keep SNPs inside region AND present in LD snplist
    ($2==C && $3>=S && $3<=E) && ($1 in keep) {
      b = $7 + 0; z = $9 + 0;
      # robust SE: skip truly zero z; else se = |beta/z|
      if (z == 0) next;
      se = b / z; if (se < 0) se = -se;
      print $1,$2,$3,$4,$5,$6,b,se
    }' "${prefix}.snplist" P1_final_clean.z > "${prefix}.z"

  # sanity check: header + ≥1 data line, exactly 8 columns
  awk 'NR==1{next} {ok=1; if(NF!=8){bad=1; exit}} END{if(!ok) exit 2; if(bad) exit 1}' "${prefix}.z" \
    || echo "!! Check ${prefix}.z (empty or wrong columns)"
done < P1_regions_250kb_fixed.bed

In [ ]:
# Drop header + normalize whitespace in every region z
for z in finemap_P1/region_*_*.z; do
  tail -n +2 "$z" | awk -v OFS=' ' '{$1=$1; print}' > "$z.tmp" && mv "$z.tmp" "$z"
done

# Rebuild every master file with n=263
for pfx in finemap_P1/region_*_*; do
  cat > "$pfx.master" <<EOF
z;ld;snp;config;cred;log;n_samples
$pfx.z;$pfx.ld;$pfx.snp;$pfx.config;$pfx.cred;$pfx.log;263
EOF
done

In [ ]:
# Prepend header everywhere
for z in finemap_P1/region_*_*.z; do
  { echo "rsid chromosome position allele1 allele2 maf beta se"; cat "$z"; } > "$z.tmp" && mv "$z.tmp" "$z"
done

# Rebuild .snp files (skip header)
for pfx in finemap_P1/region_*_*; do
  awk 'NR>1{print $1}' "$pfx.z" > "$pfx.snp"
done

# Rebuild master files (n=263)
for pfx in finemap_P1/region_*_*; do
  cat > "$pfx.master" <<EOF
z;ld;snp;config;cred;log;n_samples
$pfx.z;$pfx.ld;$pfx.snp;$pfx.config;$pfx.cred;$pfx.log;263
EOF
done

# Optional quick validation: first z-row has 8 fields
for z in finemap_P1/region_*_*.z; do
  awk 'NR==2{if(NF!=8){print "!! bad NF in",FILENAME,"NF="NF; exit 1}}' "$z" || true
done

# Run all regions
for m in finemap_P1/region_*_*.master; do
  ./finemap_v1.4_x86_64/finemap_v1.4_x86_64 --sss --in-files "$m" --n-causal-snps 3 --log
done

In [ ]:
BIM=MBBRAINS_R123-HumanCoreExome-12v1_chrall_rsq9_hrc1.1.bim
FRQ=cohort_freq.frq
ZALL=P1_final_clean.z
pfx=finemap_P1/region_9_rs78125821

awk -v FS='[ \t]+' -v OFS=' ' \
    -v BIM="$BIM" -v FRQ="$FRQ" -v Z="$ZALL" '
BEGIN{
  # --- Load GWAS Z (skip header); compute safe nonzero SE
  while ((getline line < Z) > 0){
    if (line ~ /^snp[ \t]/) continue
    split(line,f,FS)
    id=f[1]
    z_chr[id]=f[2]; z_pos[id]=f[3]; z_a1[id]=f[4]; z_a2[id]=f[5]; z_maf[id]=f[6]
    beta = f[7] + 0
    se_raw = f[8]; sub(/-.*/,"", se_raw)
    zval = f[9] + 0
    se = se_raw + 0
    if (se == 0) {
      if (zval != 0) { se = beta / zval; if (se < 0) se = -se }
      else { se = 1 }  # neutral if both beta and z are zero/missing
    }
    z_beta[id]=beta; z_se[id]=se
  }
  close(Z)

  # --- Load BIM fallback (use different array name than any scalar!)
  while ((getline line < BIM) > 0){
    split(line,bf,FS)
    id=bf[2]; bim_chr[id]=bf[1]; bim_pos[id]=bf[4]; bim_a1[id]=bf[5]; bim_a2[id]=bf[6]
  }
  close(BIM)

  # --- Load FRQ fallback MAF
  while ((getline line < FRQ) > 0){
    if (line ~ /^ *CHR[ \t]/) continue
    split(line,qf,FS)
    id=qf[2]; frq_maf[id]=qf[5]
  }
  close(FRQ)

  print "rsid","chromosome","position","allele1","allele2","maf","beta","se"
}
# --- Emit exactly in .snplist (LD) order
{
  id=$1
  if (id in z_beta) {
    print id, z_chr[id], z_pos[id], z_a1[id], z_a2[id], z_maf[id], z_beta[id], z_se[id]
  } else if (id in bim_chr) {
    maf = (id in frq_maf ? frq_maf[id] : 0.2)
    print id, bim_chr[id], bim_pos[id], bim_a1[id], bim_a2[id], maf, 0, 1
  }
}
' "$pfx.snplist" > "$pfx.z"

# Make .snp to match .z (no header)
tail -n +2 "$pfx.z" | awk '{print $1}' > "$pfx.snp"

# Sanity: all counts must match
echo -n "LD dim: "; awk 'NR==1{print NF; exit}' "$pfx.ld"
echo -n "z rows: ";   tail -n +2 "$pfx.z" | wc -l
echo -n "snp rows: "; wc -l < "$pfx.snp"

# Master (n=263)
cat > "$pfx.master" <<EOF
z;ld;snp;config;cred;log;n_samples
$pfx.z;$pfx.ld;$pfx.snp;$pfx.config;$pfx.cred;$pfx.log;263
EOF

# Run
./finemap_v1.4_x86_64/finemap_v1.4_x86_64 --sss --in-files "$pfx.master" --n-causal-snps 3 --log

In [ ]:
for pfx in finemap_P1/region_*_*; do
  # Tabify LD
  awk '{$1=$1}1' OFS='\t' "$pfx.ld" > "$pfx.ld.tmp" && mv "$pfx.ld.tmp" "$pfx.ld"

  # Ensure .z is tabbed and .snp matches
  awk 'BEGIN{OFS="\t"} NR==1{print $1,$2,$3,$4,$5,$6,$7,$8; next} {print $1,$2,$3,$4,$5,$6,$7,$8}' \
    "$pfx.z" > "$pfx.z.tmp" && mv "$pfx.z.tmp" "$pfx.z"
  tail -n +2 "$pfx.z" | cut -f1 > "$pfx.snp"

  # Sanity checks
  dim=$(awk -F'\t' 'NR==1{print NF; exit}' "$pfx.ld")
  nZ=$(tail -n +2 "$pfx.z" | wc -l)
  nS=$(wc -l < "$pfx.snp")
  if [ "$dim" -ne "$nZ" ] || [ "$nZ" -ne "$nS" ]; then
    echo "!! mismatch in $pfx  LD:$dim  z:$nZ  snp:$nS"
  fi
done

In [ ]:
# 0) choose your region prefix to test (one that failed)
pfx="finemap_P2/region_"

# 1) Make LD strictly space-delimited (no tabs/CR/blank lines)
tr '\t' ' ' < "$pfx.ld" | tr -d '\r' | sed -e 's/^ *//' -e 's/  \+/ /g' -e '/^$/d' > "$pfx.ld.tmp" \
  && mv "$pfx.ld.tmp" "$pfx.ld"

# 2) Make Z strictly 8 columns, space-delimited, with the exact header
{ 
  echo "rsid chromosome position allele1 allele2 maf beta se"
  tail -n +2 "$pfx.z" | awk -v FS='[ \t]+' -v OFS=' ' '
    {
      id=$1; chr=$2; pos=$3; a1=$4; a2=$5; maf=$6; b=$7; se=$8; z=$9
      if (maf=="" || maf=="NA") maf=0.2
      if (se=="" || se=="NA" || se==0) {
        zn = (z=="" || z=="NA") ? 0 : z+0
        se = (zn!=0) ? (b/zn) : 1
        if (se<0) se=-se
      }
      printf "%s %s %s %s %s %s %.10g %.10g\n", id, chr, pos, a1, a2, maf, b+0, se+0
    }'
} | tr -d '\r' | sed -e 's/^ *//' -e 's/  \+/ /g' > "$pfx.z.tmp" \
  && mv "$pfx.z.tmp" "$pfx.z"

# 3) SNP list = first column of Z (FINEMAP will overwrite .snp anyway, but OK)
tail -n +2 "$pfx.z" | awk '{print $1}' > "$pfx.snp"

# 4) Sanity checks using spaces (what FINEMAP actually sees)
echo -n "LD dim: "; awk '{print NF; exit}' "$pfx.ld"
echo -n "Z rows: "; tail -n +2 "$pfx.z" | wc -l
echo -n "SNPs:   "; wc -l < "$pfx.snp"

# 5) Run FINEMAP
./finemap_v1.4_x86_64/finemap_v1.4_x86_64 --sss --in-files "$pfx.master" --n-causal-snps 3 --log

In [ ]:
# P2 zfile
cat > create_P2_zfile_manual.sh << 'EOF'
#!/bin/bash
# Create Z file for P2

GWAS="P2.ADD.assoc.linear"
BIM="MBBRAINS_R123-HumanCoreExome-12v1_chrall_rsq9_hrc1.1.bim"
FRQ="cohort_freq.frq"

# Pre-load data
awk '{print $1":"$4, $2, $5, $6}' "$BIM" | tr -d '\r' > full_bim_cache.txt
awk 'NR>1 {print $2, $5}' "$FRQ" | tr -d '\r' > frq_cache.txt

awk -v OFS='\t' '
BEGIN {
    print "snp", "chrom", "pos", "a1", "a2", "maf", "beta", "se", "z"
    while ((getline line < "full_bim_cache.txt") > 0) {
        split(line, parts, " "); chrpos=parts[1]; rsid=parts[2];
        chrpos_to_rsid[chrpos]=rsid; chrpos_to_a1[chrpos]=parts[3]; chrpos_to_a2[chrpos]=parts[4];
        rsid_to_a1[rsid]=parts[3]; rsid_to_a2[rsid]=parts[4];
    }
    while ((getline line < "frq_cache.txt") > 0) {
        split(line, parts, " "); maf_by_rsid[parts[1]]=parts[2];
    }
}
NR>1 && $5=="ADD" {
    snp_id=$2; chrom=$1; pos=$3; beta=$7; se=$8; z=$11;
    sub(/-.*/, "", se);
    a1="NA"; a2="NA"; maf="NA";
    
    if (snp_id ~ /^rs/) {
        if (snp_id in rsid_to_a1) { a1=rsid_to_a1[snp_id]; a2=rsid_to_a2[snp_id]; }
        if (snp_id in maf_by_rsid) { maf=maf_by_rsid[snp_id]; }
    } else {
        gsub("chr", "", snp_id); split(snp_id, parts, /[:-]/); chr_pos=parts[1]":"parts[2];
        if (chr_pos in chrpos_to_a1) { a1=chrpos_to_a1[chr_pos]; a2=chrpos_to_a2[chr_pos]; }
        if (chr_pos in chrpos_to_rsid) { rsid=chrpos_to_rsid[chr_pos]; 
            if (rsid in maf_by_rsid) { maf=maf_by_rsid[rsid]; } }
    }
    print snp_id, chrom, pos, a1, a2, maf, beta, se, z;
}' "$GWAS" > P2.z

rm -f full_bim_cache.txt frq_cache.txt
EOF


chmod +x create_P2_zfile_manual.sh
./create_P2_zfile_manual.sh

head -n 10 P2.z
wc -l P2.z



In [ ]:
pfx="finemap_P2/region_"

tr '\t' ' ' < "$pfx.ld" | tr -d '\r' | sed -e 's/^ *//' -e 's/  \+/ /g' -e '/^$/d' > "$pfx.ld.tmp" \
  && mv "$pfx.ld.tmp" "$pfx.ld"

{ 
  echo "rsid chromosome position allele1 allele2 maf beta se"
  tail -n +2 "$pfx.z" | awk -v FS='[ \t]+' -v OFS=' ' '
    {
      id=$1; chr=$2; pos=$3; a1=$4; a2=$5; maf=$6; b=$7; se=$8; z=$9
      if (maf=="" || maf=="NA") maf=0.2
      if (se=="" || se=="NA" || se==0) {
        zn = (z=="" || z=="NA") ? 0 : z+0
        se = (zn!=0) ? (b/zn) : 1
        if (se<0) se=-se
      }
      printf "%s %s %s %s %s %s %.10g %.10g\n", id, chr, pos, a1, a2, maf, b+0, se+0
    }'
} | tr -d '\r' | sed -e 's/^ *//' -e 's/  \+/ /g' > "$pfx.z.tmp" \
  && mv "$pfx.z.tmp" "$pfx.z"

tail -n +2 "$pfx.z" | awk '{print $1}' > "$pfx.snp"

echo -n "LD dim: "; awk '{print NF; exit}' "$pfx.ld"
echo -n "Z rows: "; tail -n +2 "$pfx.z" | wc -l
echo -n "SNPs:   "; wc -l < "$pfx.snp"

./finemap_v1.4_x86_64/finemap_v1.4_x86_64 --sss --in-files "$pfx.master" --n-causal-snps 3 --log

In [ ]:
# Extract posterior signals for all regions
for L in finemap_P2/region_*_*.log_sss; do
r=${L%.log_sss}
meank=$(awk '/Post-expected # of causal SNPs/{print $NF}' "$L")
bf=$(awk '/Log10-BF of >= one causal SNP/{print $NF}' "$L")
postk=$(awk '/Post-Pr\(# of causal SNPs is k\)/{flag=1;next} flag && NF{print; if($1=="-") exit}' "$L" | tr '\n' ' ')
printf "%s\t%s\t%s\t%s\n" "$(basename "$r")" "$meank" "$bf" "$postk"
done

region_35 P2 
Error : Expected a floating point value greater than 0.0 and smaller or equal than 0.5 in line 262 column 6 of file 'finemap_P2/region_35_rs144245548.z'!

In [ ]:
plink \
--bfile MBBRAINS_R123-HumanCoreExome-12v1_chrall_rsq9_hrc1  \
--clump MBBRAINS_R123-HumanCoreExome-12v1.AVERAGEFA.assoc.linear \
--clump-p1 1e-5 \
--clump-p2 .05 \
--clump-r2 .2 \
--clump-kb 1000 \
--clump-verbose \
--clump-range glist-hg19 \
--out MBBRAINS_R123-HumanCoreExome-12v1.AVERAGEFA